In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# valid set: /content/drive/.shortcut-targets-by-id/1sVS2twm9XxgGuRNW3vW18clmRzePpKFM/dev/joint/
# train set: /content/drive/.shortcut-targets-by-id/1h2mAnKTQWMhaHmb70OqM-Jh81sG_pPLt/train/joint/

In [ ]:
# !wget "http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train"
# !wget "http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev"
# !wget "http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.test"

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/SLT-2020/phoenix14t.pami0.train','/content/')
shutil.copy('/content/drive/MyDrive/SLT-2020/phoenix14t.pami0.dev','/content/')

'/content/phoenix14t.pami0.dev'

In [ ]:
import pickle
import gzip
import torch
import os
import numpy as np
import shutil

In [ ]:
def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
valid_data = load_dataset_file("/content/phoenix14t.pami0.dev")
type(valid_data)
train_data = load_dataset_file('/content/phoenix14t.pami0.train')
type(train_data)

list

In [ ]:
print(len(valid_data))
print(len(train_data))

519
7096


In [ ]:
#valid_data[0]
train_data[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[0.0998, 0.0000, 0.1679,  ..., 0.0000, 0.0000, 0.0000],
         [4.6956, 0.0000, 0.2897,  ..., 0.0000, 0.0000, 0.0000],
         [6.8340, 0.0492, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [9.7269, 0.4355, 0.0000,  ..., 3.4021, 0.0000, 0.0000],
         [8.1934, 0.3163, 0.0000,  ..., 0.0378, 0.0000, 0.0000],
         [6.4664, 0.1839, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
#type(valid_data[0]['sign'])
type(train_data[0]['sign'])

torch.Tensor

In [ ]:
#valid_data[0]['name']

In [ ]:
# /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_12_frame_div/train/sl_gcn_output_finetune/joint'

# /content/drive/.shortcut-targets-by-id/1LiUsGee_42KZek_UKIAglrWNFS0iyRPW/4-2 shared/HRNet_Outputs_1_frame_div/train/sl_gcn_output/joint/

In [ ]:
our_train_list = os.listdir('/content/drive/.shortcut-targets-by-id/1h2mAnKTQWMhaHmb70OqM-Jh81sG_pPLt/train/joint/')
print(len(our_train_list))
our_valid_list = os.listdir('/content/drive/.shortcut-targets-by-id/1sVS2twm9XxgGuRNW3vW18clmRzePpKFM/dev/joint/')
print(len(our_valid_list))

7096
519


In [ ]:
#train_data[0]['sign'].shape

In [ ]:
source = f'/content/drive/.shortcut-targets-by-id/1h2mAnKTQWMhaHmb70OqM-Jh81sG_pPLt/train/joint/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  #print(source + name + '.npy')
  #break
  np_file = np.load(source + name + '_data_joint.npy')
  tensor_file = torch.from_numpy(np_file)  # torch.Size([11, 3, 8, 27, 1])
  #print('Shape-1: {}'.format(tensor_file.shape))
  
  #print("Original Shape: {}".format(tensor_file.shape)) # Original Shape: torch.Size([11, 3, 8, 27, 1])
  
  tensor_file = torch.squeeze(tensor_file, 4)
  #print('Shape-2: {}'.format(tensor_file.shape))

  tensor_file = torch.permute(tensor_file, (0, 2, 3, 1)) # torch.Size([11, 8, 27, 3])
  #print('Shape-3: {}'.format(tensor_file.shape))

  tensor_file = tensor_file[:,:,:,0:2] # torch.Size([11, 8, 27, 2])
  #print('Shape-4: {}'.format(tensor_file.shape))

  tensor_file = tensor_file.reshape(tensor_file.shape[0], 8, -1) # torch.Size([11, 8, 54])
  #print('Shape-5: {}'.format(tensor_file.shape))


  #tensor_file = tensor_file.reshape(tensor_file.shape[0],-1) 
  
  #tensor_file = tensor_file.mean(1) #
  #print('Shape-6: {}'.format(tensor_file.shape))

  tensor_file = tensor_file.reshape(tensor_file.shape[0], -1)

  temp_dict = sample
  temp_dict['sign'] = tensor_file

  train_set.append(temp_dict)
  count += 1
  if count % 50 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

In [ ]:
source = f'/content/drive/.shortcut-targets-by-id/1sVS2twm9XxgGuRNW3vW18clmRzePpKFM/dev/joint/'

valid_set = []
count = 0

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  #print(source + name + '.npy')
  np_file = np.load(source + name + '_data_joint.npy')
  tensor_file = torch.from_numpy(np_file)  # 
  #print("Original Shape: {}".format(tensor_file.shape))
  #break

  tensor_file = torch.squeeze(tensor_file, 4)
  tensor_file = torch.permute(tensor_file, (0, 2, 3, 1))
  tensor_file = tensor_file[:,:,:,0:2]
  tensor_file = tensor_file.reshape(tensor_file.shape[0], 8, -1)
  tensor_file = tensor_file.reshape(tensor_file.shape[0], -1)

  #tensor_file = torch.squeeze(tensor_file,1) # 
  #tensor_file = tensor_file.reshape(tensor_file.shape[0],-1) # 
  #tensor_file = tensor_file.mean(2) # 

  temp_dict = sample
  temp_dict['sign'] = tensor_file

  valid_set.append(temp_dict)
  count += 1
  if count % 20 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
type(train_set)

list

In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-2/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-2/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-2/valid_dataset.pkl.gz'

### rough

In [ ]:
!gzip -k /content/valid_dataset.pkl

In [ ]:
data = load_dataset_file('/content/valid_dataset.pkl.gz')
type(data)

list

In [ ]:
temp = valid_set[0]['sign']
temp.shape

torch.Size([3, 1, 256, 108])

In [ ]:
temp2 = torch.squeeze(temp)
temp2.shape

torch.Size([3, 256, 108])

In [ ]:
temp3 = temp2.view(3,1024,-1)
temp3.shape

torch.Size([3, 1024, 27])

In [ ]:
temp4 = temp3.mean(2)
temp4.shape

torch.Size([3, 1024])

In [ ]:
test = torch.rand(3,2,3)
test

tensor([[[0.9234, 0.6904, 0.9907],
         [0.5823, 0.0578, 0.1708]],

        [[0.0577, 0.8941, 0.1372],
         [0.0254, 0.6440, 0.1795]],

        [[0.7731, 0.3997, 0.6134],
         [0.0189, 0.4463, 0.8071]]])

In [ ]:
test.mean(2)

tensor([[0.8681, 0.2703],
        [0.3630, 0.2830],
        [0.5954, 0.4241]])

In [ ]:
temp.shape

torch.Size([3, 1, 256, 108])

In [ ]:
temp5 = temp.view(3,1,1024,-1)
temp5.shape

torch.Size([3, 1, 1024, 27])

In [ ]:
temp.shape

torch.Size([3, 1, 256, 108])

In [ ]:
temp.mean(3).mean(1).shape

torch.Size([3, 256])

### back again

In [ ]:
temp_list = os.listdir('/content/drive/.shortcut-targets-by-id/1LiUsGee_42KZek_UKIAglrWNFS0iyRPW/4-2 shared/HRNet_Outputs_1_frame_div/dev/')

In [ ]:
temp_list

['npy_1_frame', 'sl_gcn_output', 'pkl', 'joint', 'csv']

### r2+1d

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/R2+1D/Attempt-2/valid_r21d.zip','/content/valid/')

In [ ]:
!mkdir valid_dataset

In [ ]:
source = f'/content/valid/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([27, 512])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([8, 512])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([11, 512])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([26, 512])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([28, 512])


In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/R2+1D/Attempt-2/train_r21d.zip','/content/train/')

In [ ]:
len(os.listdir('/content/train/'))

7096

In [ ]:
source = f'/content/train/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 1000 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([16, 512])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([25, 512])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([6, 512])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([22, 512])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([31, 512])
count: 6000, sample: 10December_2009_Thursday_tagesschau-7491, shape: torch.Size([14, 512])
count: 7000, sample: 15July_2011_Friday_tagesschau-8722, shape: torch.Size([26, 512])


In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/R2+1D/Attempt-2/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/R2+1D/Attempt-2/')

'/content/drive/MyDrive/SLT-2020/CNN_Dataset/R2+1D/Attempt-2/valid_dataset.pkl.gz'

### i3d

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-7/valid_i3d.zip','/content/valid/')

In [ ]:
source = f'/content/valid/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([38, 1024])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([9, 1024])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([14, 1024])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([36, 1024])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([39, 1024])


In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-7/train_i3d.zip','/content/train/')

In [ ]:
len(os.listdir('/content/train/kaggle/working/train_dataset/'))

7096

In [ ]:
source = f'/content/train/kaggle/working/train_dataset/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 01November_2010_Monday_tagesschau-132, shape: torch.Size([21, 1024])
count: 200, sample: 12July_2010_Monday_heute-256, shape: torch.Size([61, 1024])
count: 300, sample: 12July_2010_Monday_tagesschau-381, shape: torch.Size([31, 1024])
count: 400, sample: 20October_2011_Thursday_tagesschau-504, shape: torch.Size([21, 1024])
count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([47, 1024])
count: 600, sample: 09November_2010_Tuesday_tagesschau-753, shape: torch.Size([18, 1024])
count: 700, sample: 29January_2013_Tuesday_tagesschau-883, shape: torch.Size([25, 1024])
count: 800, sample: 29October_2009_Thursday_tagesschau-1012, shape: torch.Size([26, 1024])
count: 900, sample: 22October_2009_Thursday_tagesschau-1128, shape: torch.Size([22, 1024])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([20, 1024])
count: 1100, sample: 10January_2011_Monday_heute-1374, shape: torch.Size([66, 1024])
count: 1200, sample: 17February_2010_Wednes

In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-7/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-7/')

'/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-7/valid_dataset.pkl.gz'

In [ ]:
!unzip -q /content/01April_2010_Thursday_heute-6697.zip

In [ ]:
import cv2
import os

image_folder = '/content/01April_2010_Thursday_heute-6697/'
video_name = 'output.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

#cv2.destroyAllWindows()
video.release()

### i3d (6+8+12)

In [ ]:
import shutil
#shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-3/train_i3d.zip','/content/train_8/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-4/train_i3d.zip','/content/train_12/')
#shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-5/train_i3d.zip','/content/train_6/')

In [ ]:
import os
print(len(os.listdir('/content/train_6/train_i3d/')))
print(len(os.listdir('/content/train_8/')))
print(len(os.listdir('/content/train_12/train_dataset/')))

7096
7096
7096


In [ ]:
!mkdir train_dataset

In [ ]:
import os
import torch

source = '/content/train_6/train_i3d/' # has most number of segments
target = '/content/train_dataset/'

print("Number of 6 Frame Samples: {}".format(len(os.listdir(source))))
count = 0

for sample in sorted(os.listdir(source)):
  path = source + sample
  tensor6 = torch.load(path, map_location = torch.device('cpu'))
  #print('6 shape: {}'.format(tensor6.shape))
  lst6 = tensor6.tolist()
  target_len = len(lst6)

  tensor8 = torch.load('/content/train_8/' + sample , map_location = torch.device('cpu'))
  #print('8 shape: {}'.format(tensor8.shape))
  lst8 = tensor8.tolist()
  while len(lst8) < target_len:
    lst8.append(lst8[-1])

  tensor12 = torch.load('/content/train_12/train_dataset/' + sample , map_location = torch.device('cpu'))
  #print('12 shape: {}'.format(tensor12.shape))
  lst12 = tensor12.tolist()
  while len(lst12) < target_len:
    lst12.append(lst12[-1])

  new6 = torch.Tensor(lst6) 
  new8 = torch.Tensor(lst8)
  new12 = torch.Tensor(lst12)

  #print('6 shape after: {}'.format(new6.shape))
  #print('8 shape after: {}'.format(new8.shape))
  #print('12 shape after: {}'.format(new12.shape))
  
  out = torch.cat([new6, new8, new12], dim = 1)
  torch.save(out, target + sample, _use_new_zipfile_serialization = False)
  count += 1

  if count % 100 == 0:
    print('Count: {}, Sample: {}, Shape: {}'.format(count, sample, out.shape))

print('Output: {}'.format(len(os.listdir(target))))  

Number of 6 Frame Samples: 7096
Count: 100, Sample: 01July_2009_Wednesday_tagesschau-4553.pt, Shape: torch.Size([18, 3072])
Count: 200, Sample: 01November_2010_Monday_heute-1722.pt, Shape: torch.Size([12, 3072])
Count: 300, Sample: 02December_2010_Thursday_tagesschau-3634.pt, Shape: torch.Size([12, 3072])
Count: 400, Sample: 02November_2010_Tuesday_heute-2756.pt, Shape: torch.Size([9, 3072])
Count: 500, Sample: 03August_2010_Tuesday_heute-7614.pt, Shape: torch.Size([15, 3072])
Count: 600, Sample: 03July_2009_Friday_tagesschau-2005.pt, Shape: torch.Size([17, 3072])
Count: 700, Sample: 03October_2012_Wednesday_tagesschau-7663.pt, Shape: torch.Size([10, 3072])
Count: 800, Sample: 04January_2010_Monday_tagesschau-8549.pt, Shape: torch.Size([18, 3072])
Count: 900, Sample: 04May_2010_Tuesday_tagesschau-7846.pt, Shape: torch.Size([21, 3072])
Count: 1000, Sample: 05December_2009_Saturday_tagesschau-4699.pt, Shape: torch.Size([21, 3072])
Count: 1100, Sample: 05July_2009_Sunday_tagesschau-4452.p

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-3/valid_i3d.zip','/content/valid_8/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-4/valid_i3d.zip','/content/valid_12/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-5/valid_i3d.zip','/content/valid_6/')

In [ ]:
import os
print(len(os.listdir('/content/valid_6/')))
print(len(os.listdir('/content/valid_8/')))
print(len(os.listdir('/content/valid_12/')))

519
519
519


In [ ]:
!mkdir valid_dataset

In [ ]:
import os
import torch

source = '/content/valid_6/' # has most number of segments
target = '/content/valid_dataset/'

print("Number of 6 Frame Samples: {}".format(len(os.listdir(source))))
count = 0

for sample in sorted(os.listdir(source)):
  path = source + sample
  tensor6 = torch.load(path, map_location = torch.device('cpu'))
  #print('6 shape: {}'.format(tensor6.shape))
  lst6 = tensor6.tolist()
  target_len = len(lst6)

  tensor8 = torch.load('/content/valid_8/' + sample , map_location = torch.device('cpu'))
  #print('8 shape: {}'.format(tensor8.shape))
  lst8 = tensor8.tolist()
  while len(lst8) < target_len:
    lst8.append(lst8[-1])

  tensor12 = torch.load('/content/valid_12/' + sample , map_location = torch.device('cpu'))
  #print('12 shape: {}'.format(tensor12.shape))
  lst12 = tensor12.tolist()
  while len(lst12) < target_len:
    lst12.append(lst12[-1])

  new6 = torch.Tensor(lst6) 
  new8 = torch.Tensor(lst8)
  new12 = torch.Tensor(lst12)

  #print('6 shape after: {}'.format(new6.shape))
  #print('8 shape after: {}'.format(new8.shape))
  #print('12 shape after: {}'.format(new12.shape))
  
  out = torch.cat([new6, new8, new12], dim = 1)
  torch.save(out, target + sample, _use_new_zipfile_serialization = False)
  count += 1

  if count % 50 == 0:
    print('Count: {}, Sample: {}, Shape: {}'.format(count, sample, out.shape))

print('Output: {}'.format(len(os.listdir(target))))  

Number of 6 Frame Samples: 519
Count: 50, Sample: 03October_2012_Wednesday_tagesschau-7651.pt, Shape: torch.Size([17, 3072])
Count: 100, Sample: 06July_2011_Wednesday_tagesschau-6870.pt, Shape: torch.Size([26, 3072])
Count: 150, Sample: 09February_2011_Wednesday_heute-7538.pt, Shape: torch.Size([27, 3072])
Count: 200, Sample: 12February_2010_Friday_tagesschau-126.pt, Shape: torch.Size([18, 3072])
Count: 250, Sample: 16December_2009_Wednesday_tagesschau-5918.pt, Shape: torch.Size([19, 3072])
Count: 300, Sample: 20July_2010_Tuesday_tagesschau-7877.pt, Shape: torch.Size([20, 3072])
Count: 350, Sample: 23April_2010_Friday_tagesschau-3668.pt, Shape: torch.Size([17, 3072])
Count: 400, Sample: 25April_2010_Sunday_tagesschau-4952.pt, Shape: torch.Size([9, 3072])
Count: 450, Sample: 27September_2009_Sunday_tagesschau-2169.pt, Shape: torch.Size([27, 3072])
Count: 500, Sample: 30November_2009_Monday_tagesschau-5755.pt, Shape: torch.Size([18, 3072])
Output: 519


In [ ]:
source = f'/content/valid_dataset/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([27, 3072])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([8, 3072])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([11, 3072])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([26, 3072])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([28, 3072])


In [ ]:
source = f'/content/train_dataset/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 01November_2010_Monday_tagesschau-132, shape: torch.Size([16, 3072])
count: 200, sample: 12July_2010_Monday_heute-256, shape: torch.Size([43, 3072])
count: 300, sample: 12July_2010_Monday_tagesschau-381, shape: torch.Size([23, 3072])
count: 400, sample: 20October_2011_Thursday_tagesschau-504, shape: torch.Size([16, 3072])
count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([33, 3072])
count: 600, sample: 09November_2010_Tuesday_tagesschau-753, shape: torch.Size([14, 3072])
count: 700, sample: 29January_2013_Tuesday_tagesschau-883, shape: torch.Size([19, 3072])
count: 800, sample: 29October_2009_Thursday_tagesschau-1012, shape: torch.Size([20, 3072])
count: 900, sample: 22October_2009_Thursday_tagesschau-1128, shape: torch.Size([17, 3072])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([16, 3072])
count: 1100, sample: 10January_2011_Monday_heute-1374, shape: torch.Size([46, 3072])
count: 1200, sample: 17February_2010_Wednes

In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-6/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-6/')

'/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-6/valid_dataset.pkl.gz'

### HRNet 133 keypoints

In [ ]:
import shutil
import os
import numpy as np
import torch
from pathlib import Path

In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/train.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/valid.zip','/content/valid/')

In [ ]:
#shutil.rmtree('/content/train_after_discard')
#shutil.rmtree('/content/valid_after_discard')

In [ ]:
def discard_frames(source, dest):
  print("Number of Input Samples: {}".format(len(os.listdir(source))))

  # check if destination exists, create otherwise
  path = Path(dest)
  if not path.exists():
    os.makedirs(path) 

  count = 0
  # iterate through each sample
  for sample in sorted(os.listdir(source)):
    data = np.load(source + sample)
    
    # list to store new data
    lst = []
    discard_count = 0
    # iterate through each frame
    for idx in range(data.shape[0]):
      # if all hand keypoints doesn't average to
      # more than 0.7 confidence score, discard the frame
      conf_mean = np.mean(data[idx, 91:133, 2]) 
      if conf_mean < 0.7:
        discard_count += 1
        continue
      lst.append(data[idx, :, 0:2])

    if discard_count == data.shape[0]:
      tensor = torch.from_numpy(data)
      torch.save(tensor, dest + sample[:-4] + '.pt', _use_new_zipfile_serialization = False)    
    else:
      new_data = np.array(lst)
      tensor = torch.from_numpy(new_data)
      torch.save(tensor, dest + sample[:-4] + '.pt', _use_new_zipfile_serialization = False)
    
    count += 1
    if count % 500 == 0:
      print("Sample: {} , Shape: {}, Discarded: {}".format(count, tensor.shape, discard_count))

  print("Number of Output Samples: {}".format(len(os.listdir(dest))))

In [ ]:
# train set
discard_frames('/content/train/train/', '/content/train_after_discard/')

Number of Input Samples: 7096
Sample: 500 , Shape: torch.Size([73, 133, 2]), Discarded: 17
Sample: 1000 , Shape: torch.Size([89, 133, 2]), Discarded: 33
Sample: 1500 , Shape: torch.Size([71, 133, 2]), Discarded: 43
Sample: 2000 , Shape: torch.Size([61, 133, 2]), Discarded: 66
Sample: 2500 , Shape: torch.Size([4, 133, 2]), Discarded: 79
Sample: 3000 , Shape: torch.Size([22, 133, 2]), Discarded: 75
Sample: 3500 , Shape: torch.Size([40, 133, 2]), Discarded: 49
Sample: 4000 , Shape: torch.Size([23, 133, 2]), Discarded: 22
Sample: 4500 , Shape: torch.Size([147, 133, 2]), Discarded: 102
Sample: 5000 , Shape: torch.Size([22, 133, 2]), Discarded: 42
Sample: 5500 , Shape: torch.Size([11, 133, 2]), Discarded: 77
Sample: 6000 , Shape: torch.Size([123, 133, 2]), Discarded: 35
Sample: 6500 , Shape: torch.Size([1, 133, 2]), Discarded: 105
Sample: 7000 , Shape: torch.Size([89, 133, 2]), Discarded: 12
Number of Output Samples: 7096


In [ ]:
# valid set
discard_frames('/content/valid/dev/', '/content/valid_after_discard/')

Number of Input Samples: 519
Sample: 500 , Shape: torch.Size([46, 133, 2]), Discarded: 60
Number of Output Samples: 519


In [ ]:
def norm_segment(source, dest): # window_size):
  print("Number of Input Samples: {}".format(len(os.listdir(source))))

  # check if destination exists, create otherwise
  path = Path(dest)
  if not path.exists():
    os.makedirs(path) 

  count = 0
  # iterate through each sample
  for sample in sorted(os.listdir(source)):
    data = torch.load(source + sample)
    #frame_count = data.shape[0]

    '''
    if frame_count % window_size:
      # extracting last frame that should be repeated
      to_stack = data[frame_count - 1, :, :].unsqueeze(0)
      num = window_size - (frame_count % window_size)
      to_stack = to_stack.repeat(num, 1, 1)
      data = torch.cat([data, to_stack])
    '''
    
    lst = []
    # iterate through each frame
    for idx in range(data.shape[0]):
      x = data[idx, :, 0]
      y = data[idx, :, 1]
      mean_x = torch.mean(x)
      mean_y = torch.mean(y)
      std_x = torch.std(x)
      std_y = torch.std(y)
      norm_x = (x-mean_x)/std_x
      norm_y = (y-mean_y)/std_y
      coord = torch.cat([norm_x, norm_y])

      lst.append(coord)
      '''
      i = idx + 1
      if idx == 0:
        out = coord
      elif i % window_size == 0:
        out = torch.cat([out, coord])
        lst.append(out)
      else:
        out = torch.cat([out, coord])
      '''

    output = torch.stack(lst)
    torch.save(output, dest + sample, _use_new_zipfile_serialization = False)
    count += 1
    if count % 500 == 0:
      print("Sample: {}, Shape: {}".format(count, output.shape))

  print("Number of Output Samples: {}".format(len(os.listdir(dest))))

In [ ]:
#shutil.rmtree('/content/train_data')

In [ ]:
norm_segment('/content/train_after_discard/','/content/train_data/')

Number of Input Samples: 7096
Sample: 500, Shape: torch.Size([73, 266])
Sample: 1000, Shape: torch.Size([89, 266])
Sample: 1500, Shape: torch.Size([71, 266])
Sample: 2000, Shape: torch.Size([61, 266])
Sample: 2500, Shape: torch.Size([4, 266])
Sample: 3000, Shape: torch.Size([22, 266])
Sample: 3500, Shape: torch.Size([40, 266])
Sample: 4000, Shape: torch.Size([23, 266])
Sample: 4500, Shape: torch.Size([147, 266])
Sample: 5000, Shape: torch.Size([22, 266])
Sample: 5500, Shape: torch.Size([11, 266])
Sample: 6000, Shape: torch.Size([123, 266])
Sample: 6500, Shape: torch.Size([1, 266])
Sample: 7000, Shape: torch.Size([89, 266])
Number of Output Samples: 7096


In [ ]:
norm_segment('/content/valid_after_discard/','/content/valid_data/')

Number of Input Samples: 519
Sample: 500, Shape: torch.Size([46, 266])
Number of Output Samples: 519


In [ ]:
source = f'/content/train_data/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 500 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([177, 266])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([56, 266])
count: 1500, sample: 15February_2010_Monday_tagesschau-1850, shape: torch.Size([42, 266])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([124, 266])
count: 2500, sample: 21April_2010_Wednesday_heute-3116, shape: torch.Size([42, 266])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([15, 266])
count: 3500, sample: 23May_2010_Sunday_tagesschau-4342, shape: torch.Size([82, 266])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([99, 266])
count: 4500, sample: 18November_2009_Wednesday_tagesschau-5589, shape: torch.Size([36, 266])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([31, 266])
count: 5500, sample: 10May_2010_Monday_tagesschau-6860, shape: torch.Size([112, 266])
count: 6000, sample: 10December_2009_Thursd

In [ ]:
source = f'/content/valid_data/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([59, 266])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([10, 266])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([45, 266])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([33, 266])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([66, 266])


In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-2/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-2/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-2/valid_dataset.pkl.gz'

### HRNet 133 keypoints - Part-2

In [ ]:
import shutil
import os
import numpy as np
import torch
from pathlib import Path

In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/train.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/valid.zip','/content/valid/')

In [ ]:
#shutil.rmtree('/content/train_after_discard')
#shutil.rmtree('/content/valid_after_discard')
#shutil.rmtree('/content/train_data')
#shutil.rmtree('/content/valid_data')

In [ ]:
def discard_frames(source, dest):
  print("Number of Input Samples: {}".format(len(os.listdir(source))))

  # check if destination exists, create otherwise
  path = Path(dest)
  if not path.exists():
    os.makedirs(path) 

  count = 0
  # iterate through each sample
  for sample in sorted(os.listdir(source)):
    data = np.load(source + sample)
    data = torch.from_numpy(data)
    
    # list to store new data
    lst = []

    # iterate through each frame
    for idx in range(data.shape[0]):
      new_data = torch.cat([data[idx, 0:12, 0:2], data[idx, 23:-1, 0:2]])
      lst.append(new_data)

    tensor = torch.stack(lst)
    torch.save(tensor, dest + sample[:-4] + '.pt', _use_new_zipfile_serialization = False)    
    
    count += 1
    if count % 500 == 0:
      print("Sample:{} , Shape: {}".format(count, tensor.shape))

  print("Number of Output Samples: {}".format(len(os.listdir(dest))))

In [ ]:
# train set
discard_frames('/content/train/train/', '/content/train_after_discard/')

Number of Input Samples: 7096
Sample:500 , Shape: torch.Size([90, 121, 2])
Sample:1000 , Shape: torch.Size([122, 121, 2])
Sample:1500 , Shape: torch.Size([114, 121, 2])
Sample:2000 , Shape: torch.Size([127, 121, 2])
Sample:2500 , Shape: torch.Size([83, 121, 2])
Sample:3000 , Shape: torch.Size([97, 121, 2])
Sample:3500 , Shape: torch.Size([89, 121, 2])
Sample:4000 , Shape: torch.Size([45, 121, 2])
Sample:4500 , Shape: torch.Size([249, 121, 2])
Sample:5000 , Shape: torch.Size([64, 121, 2])
Sample:5500 , Shape: torch.Size([88, 121, 2])
Sample:6000 , Shape: torch.Size([158, 121, 2])
Sample:6500 , Shape: torch.Size([106, 121, 2])
Sample:7000 , Shape: torch.Size([101, 121, 2])
Number of Output Samples: 7096


In [ ]:
# valid set
discard_frames('/content/valid/dev/', '/content/valid_after_discard/')

Number of Input Samples: 519
Sample:500 , Shape: torch.Size([106, 121, 2])
Number of Output Samples: 519


In [ ]:
def norm_segment(source, dest): # window_size):
  print("Number of Input Samples: {}".format(len(os.listdir(source))))

  # check if destination exists, create otherwise
  path = Path(dest)
  if not path.exists():
    os.makedirs(path) 

  count = 0
  # iterate through each sample
  for sample in sorted(os.listdir(source)):
    data = torch.load(source + sample)
    #frame_count = data.shape[0]

    '''
    if frame_count % window_size:
      # extracting last frame that should be repeated
      to_stack = data[frame_count - 1, :, :].unsqueeze(0)
      num = window_size - (frame_count % window_size)
      to_stack = to_stack.repeat(num, 1, 1)
      data = torch.cat([data, to_stack])
    '''
    
    lst = []
    # iterate through each frame
    for idx in range(data.shape[0]):
      x = data[idx, :, 0]
      y = data[idx, :, 1]
      mean_x = torch.mean(x)
      mean_y = torch.mean(y)
      std_x = torch.std(x)
      std_y = torch.std(y)
      norm_x = (x-mean_x)/std_x
      norm_y = (y-mean_y)/std_y
      coord = torch.cat([norm_x, norm_y])

      lst.append(coord)
      '''
      i = idx + 1
      if idx == 0:
        out = coord
      elif i % window_size == 0:
        out = torch.cat([out, coord])
        lst.append(out)
      else:
        out = torch.cat([out, coord])
      '''

    output = torch.stack(lst)
    torch.save(output, dest + sample, _use_new_zipfile_serialization = False)
    count += 1
    if count % 500 == 0:
      print("Sample: {}, Shape: {}".format(count, output.shape))

  print("Number of Output Samples: {}".format(len(os.listdir(dest))))

In [ ]:
#shutil.rmtree('/content/train_data')

In [ ]:
norm_segment('/content/train_after_discard/','/content/train_data/')

Number of Input Samples: 7096
Sample: 500, Shape: torch.Size([90, 242])
Sample: 1000, Shape: torch.Size([122, 242])
Sample: 1500, Shape: torch.Size([114, 242])
Sample: 2000, Shape: torch.Size([127, 242])
Sample: 2500, Shape: torch.Size([83, 242])
Sample: 3000, Shape: torch.Size([97, 242])
Sample: 3500, Shape: torch.Size([89, 242])
Sample: 4000, Shape: torch.Size([45, 242])
Sample: 4500, Shape: torch.Size([249, 242])
Sample: 5000, Shape: torch.Size([64, 242])
Sample: 5500, Shape: torch.Size([88, 242])
Sample: 6000, Shape: torch.Size([158, 242])
Sample: 6500, Shape: torch.Size([106, 242])
Sample: 7000, Shape: torch.Size([101, 242])
Number of Output Samples: 7096


In [ ]:
norm_segment('/content/valid_after_discard/','/content/valid_data/')

Number of Input Samples: 519
Sample: 500, Shape: torch.Size([106, 242])
Number of Output Samples: 519


In [ ]:
source = f'/content/train_data/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 500 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([198, 242])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([92, 242])
count: 1500, sample: 15February_2010_Monday_tagesschau-1850, shape: torch.Size([93, 242])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([150, 242])
count: 2500, sample: 21April_2010_Wednesday_heute-3116, shape: torch.Size([85, 242])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([31, 242])
count: 3500, sample: 23May_2010_Sunday_tagesschau-4342, shape: torch.Size([130, 242])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([131, 242])
count: 4500, sample: 18November_2009_Wednesday_tagesschau-5589, shape: torch.Size([91, 242])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([183, 242])
count: 5500, sample: 10May_2010_Monday_tagesschau-6860, shape: torch.Size([208, 242])
count: 6000, sample: 10December_2009_Thu

In [ ]:
source = f'/content/valid_data/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([162, 242])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([45, 242])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([65, 242])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([156, 242])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([166, 242])


In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-3/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-3/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/Attempt-3/valid_dataset.pkl.gz'

### rough-2

In [ ]:
l = os.listdir('/content/train_after_discard/')
l[34]

'28January_2010_Thursday_tagesschau-2811.npy.pt'

In [ ]:
tt = torch.load('/content/train_after_discard/01August_2011_Monday_heute-4849.pt')
tt.shape

torch.Size([0])

In [ ]:
print(42 % 6)

0


In [ ]:
if 0:
  print('wut')

In [ ]:
temp = np.load('/content/train/train/01April_2010_Thursday_heute-6694.npy')
temp.shape

(53, 133, 3)

In [ ]:
temp[0, :, 2]

array([0.9638047 , 0.9590918 , 0.9573112 , 0.9016408 , 0.9116309 ,
       0.8409655 , 0.87791747, 0.8872658 , 0.88564247, 0.9322082 ,
       0.90770936, 0.66431105, 0.68250203, 0.04623652, 0.04578356,
       0.1501307 , 0.142673  , 0.35242778, 0.34163466, 0.3940376 ,
       0.30609155, 0.28177425, 0.35224986, 0.90631974, 0.9588274 ,
       0.9575112 , 0.9390445 , 0.9433827 , 0.97055334, 0.9808987 ,
       0.9656267 , 0.9717074 , 0.97239256, 0.97122204, 0.9636855 ,
       0.9577147 , 0.9402936 , 0.9639468 , 0.9581839 , 0.8942652 ,
       0.95807767, 0.97466797, 0.9797343 , 0.95632875, 0.92680466,
       0.92267716, 0.94105065, 0.95227206, 0.9510535 , 0.958421  ,
       0.9583516 , 0.9992411 , 1.00329   , 0.95539653, 0.97395766,
       1.0075103 , 1.004766  , 1.0023639 , 0.98463166, 0.9477819 ,
       0.95317113, 0.94642735, 0.9753853 , 1.0023267 , 0.99810696,
       0.9804976 , 0.9632288 , 0.9555929 , 0.9516536 , 0.97590214,
       1.0004263 , 0.9785799 , 0.9891299 , 0.9823488 , 0.97161

In [ ]:
# 92-133
len(temp[0, 91:133, 2])

42

In [ ]:
np.mean(temp[0, 91:133, 2])

0.83472633

In [ ]:
sum = 0
num = 42
for i in range(91, 133):
  sum += temp[0][i][2]
print(sum/num)  

0.8347263520672208


In [ ]:
temp[0,:].shape

(133, 3)

In [ ]:
import torch
t = torch.randn((5, 4, 3))
t.shape

torch.Size([5, 4, 3])

In [ ]:
to_stack = t[4, :, :].unsqueeze(0)
print(to_stack.shape)
to_to_stack = to_stack.repeat(7, 1, 1)
print(to_to_stack.shape)

torch.Size([1, 4, 3])
torch.Size([7, 4, 3])


In [ ]:
to_stack

tensor([[[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]]])

In [ ]:
to_to_stack

tensor([[[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],
         [ 1.5667, -0.4470, -0.2487],
         [ 2.5866, -0.5030, -0.8694]],

        [[ 0.5719,  0.5784,  0.0804],
         [ 0.4905,  0.4057, -0.8036],


In [ ]:
tt = torch.cat([t, t[4, :, :]*7])
tt.shape

RuntimeError: ignored

In [ ]:
lst = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
print(len(lst))
lst = lst[4:]
print(len(lst))

16
12


In [ ]:
lst

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

### i3d + joint slgcn

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-3/train_dataset.pkl.gz','/content/')
shutil.copy('/content/drive/MyDrive/SLT-2020/CNN_Dataset/I3D/Attempt-3/valid_dataset.pkl.gz','/content/')

'/content/valid_dataset.pkl.gz'

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/train_dataset.pkl.gz','/content/joint/')
shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/valid_dataset.pkl.gz','/content/joint/')

'/content/joint/valid_dataset.pkl.gz'

In [ ]:
i3d_train = load_dataset_file('/content/train_dataset.pkl.gz')
print(type(i3d_train))
i3d_valid = load_dataset_file('/content/valid_dataset.pkl.gz')
print(type(i3d_valid))

<class 'list'>
<class 'list'>


In [ ]:
joint_train = load_dataset_file('/content/joint/train_dataset.pkl.gz')
print(type(joint_train))
joint_valid = load_dataset_file('/content/joint/valid_dataset.pkl.gz')
print(type(joint_valid))

<class 'list'>
<class 'list'>


In [ ]:
i3d_train[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0455, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0215, 0.0000],
         [0.0656, 0.0765, 0.0241,  ..., 0.0000, 0.1074, 0.0000],
         ...,
         [0.0000, 0.0145, 0.0000,  ..., 0.0000, 0.0449, 0.0000],
         [0.2525, 0.0000, 0.0007,  ..., 0.0000, 0.0078, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0605, 0.0000]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
i3d_train[0]['sign'].shape

torch.Size([11, 1024])

In [ ]:
joint_train[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[2.3709e-01, 6.1418e-01, 4.9700e-01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.0196e-01, 1.1470e+00,
          9.3762e-01],
         [2.7983e-03, 0.0000e+00, 1.0097e-02,  ..., 1.4955e-01, 1.0444e-01,
          2.7775e-01],
         ...,
         [0.0000e+00, 0.0000e+00, 8.4339e-03,  ..., 1.1277e-01, 1.6648e-01,
          9.8992e-02],
         [0.0000e+00, 0.0000e+00, 3.7642e-04,  ..., 1.6213e-01, 4.3711e-02,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.2813e-01, 1.4803e+00,
          5.3173e-01]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
joint_train[0]['sign'].shape

torch.Size([11, 1152])

In [ ]:
len(i3d_train)

7096

In [ ]:
new_train = i3d_train
new_valid = i3d_valid

In [ ]:
import torch
for i in range(len(i3d_train)):
  new_tensor = torch.cat([i3d_train[i]['sign'], joint_train[i]['sign']], axis = 1)
  new_train[i]['sign'] = new_tensor

for i in range(len(i3d_valid)):
  new_tensor = torch.cat([i3d_valid[i]['sign'], joint_valid[i]['sign']], axis = 1)
  new_valid[i]['sign'] = new_tensor

In [ ]:
new_train[0]['sign'].shape

torch.Size([11, 2176])

In [ ]:
new_valid[0]['sign'].shape

torch.Size([6, 2176])

In [ ]:
print(len(new_train))
print(len(new_valid))

7096
519


In [ ]:
with open('/content/new/valid_dataset.pkl', 'wb') as f:
  pickle.dump(new_valid, f)
with open('/content/new/train_dataset.pkl', 'wb') as f:
  pickle.dump(new_train, f)

In [ ]:
!gzip -k /content/new/valid_dataset.pkl
!gzip -k /content/new/train_dataset.pkl

In [ ]:
shutil.copy('/content/new/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/Combined/')
shutil.copy('/content/new/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/CNN_Dataset/Combined/')

'/content/drive/MyDrive/SLT-2020/CNN_Dataset/Combined/valid_dataset.pkl.gz'

### Augmentation in 8 only joint finetuned dataset

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-2/train_dataset.pkl.gz','/content/')
#shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-2/valid_dataset.pkl.gz','/content/')

'/content/train_dataset.pkl.gz'

In [ ]:
#ori_valid_set = load_dataset_file("/content/valid_dataset.pkl.gz")
ori_train_set = load_dataset_file('/content/train_dataset.pkl.gz')
type(ori_train_set)

list

In [ ]:
#new_valid_set = ori_valid_set.copy()
new_train_set = ori_train_set.copy()

In [ ]:
print(len(new_train_set))
#print(len(new_valid_set))

7096


In [ ]:
new_train_set[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[ 99.0894,  56.3203, 137.5529,  ..., 153.3950,  53.2995, 155.2265],
         [104.5842,  61.8151, 137.5529,  ..., 114.9314,  34.9835, 100.2787],
         [104.5842,  65.4783, 137.5529,  ..., 144.2370,  55.1311, 146.0686],
         ...,
         [102.7526,  69.1415, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  70.9731, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  74.6363, 133.8898,  ..., 255.9644,  58.7943, 257.7960]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
ori_train_set[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[ 99.0894,  56.3203, 137.5529,  ..., 153.3950,  53.2995, 155.2265],
         [104.5842,  61.8151, 137.5529,  ..., 114.9314,  34.9835, 100.2787],
         [104.5842,  65.4783, 137.5529,  ..., 144.2370,  55.1311, 146.0686],
         ...,
         [102.7526,  69.1415, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  70.9731, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  74.6363, 133.8898,  ..., 255.9644,  58.7943, 257.7960]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
for idx in range(len(ori_train_set)):
  sign_tensor = ori_train_set[idx]['sign']
  # add 1 to every value
  aug_one = sign_tensor + 1.0
  # subtract 1 from every value
  aug_two = sign_tensor - 1.0
  # add 2 to every value
  aug_three = sign_tensor + 2.0
  # subtract 2 from every value
  aug_four = sign_tensor - 2.0
 
  name = ori_train_set[idx]['name']
  
  aug_sample_1 =  ori_train_set[idx].copy()
  aug_sample_1['name'] = name + '-1'
  aug_sample_1['sign'] = aug_one

  aug_sample_2 =  ori_train_set[idx].copy()
  aug_sample_2['name'] = name + '-2'
  aug_sample_2['sign'] = aug_two

  aug_sample_3 =  ori_train_set[idx].copy()
  aug_sample_3['name'] = name + '-3'
  aug_sample_3['sign'] = aug_three

  aug_sample_4 =  ori_train_set[idx].copy()
  aug_sample_4['name'] = name + '-4'
  aug_sample_4['sign'] = aug_four

  new_train_set.append(aug_sample_1)
  new_train_set.append(aug_sample_2)
  new_train_set.append(aug_sample_3)
  new_train_set.append(aug_sample_4)

  if idx % 1000 == 0:
    print("count: {}".format(idx))

count: 0
count: 1000
count: 2000
count: 3000
count: 4000
count: 5000
count: 6000
count: 7000


In [ ]:
print(len(new_train_set))
#print(len(new_valid_set))

35480


In [ ]:
print(len(ori_train_set))
#print(len(ori_valid_set))

7096


In [ ]:
new_train_set[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[ 99.0894,  56.3203, 137.5529,  ..., 153.3950,  53.2995, 155.2265],
         [104.5842,  61.8151, 137.5529,  ..., 114.9314,  34.9835, 100.2787],
         [104.5842,  65.4783, 137.5529,  ..., 144.2370,  55.1311, 146.0686],
         ...,
         [102.7526,  69.1415, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  70.9731, 133.8898,  ..., 255.9644,  58.7943, 257.7960],
         [ 97.2578,  74.6363, 133.8898,  ..., 255.9644,  58.7943, 257.7960]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
new_train_set[7096]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1-1',
 'sign': tensor([[100.0894,  57.3203, 138.5529,  ..., 154.3950,  54.2995, 156.2265],
         [105.5842,  62.8151, 138.5529,  ..., 115.9314,  35.9835, 101.2787],
         [105.5842,  66.4783, 138.5529,  ..., 145.2370,  56.1311, 147.0686],
         ...,
         [103.7526,  70.1415, 134.8898,  ..., 256.9644,  59.7943, 258.7960],
         [ 98.2578,  71.9731, 134.8898,  ..., 256.9644,  59.7943, 258.7960],
         [ 98.2578,  75.6363, 134.8898,  ..., 256.9644,  59.7943, 258.7960]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
new_train_set[7100]

{'gloss': 'ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG KOENNEN',
 'name': 'train/11August_2010_Wednesday_tagesschau-4-1',
 'sign': tensor([[101.9210,  68.3099, 134.8898,  ..., 200.1849,  70.7839, 209.3429],
         [107.4158,  71.9731, 136.7213,  ..., 161.7214,  17.6675, 159.8898],
         [103.7526,  77.4679, 138.5529,  ..., 147.0686,  65.2891, 136.0790],
         ...,
         [122.0686,  66.4783, 136.7213,  ..., 152.5634,  61.6259, 163.5530],
         [118.4054,  71.9731, 136.7213,  ..., 234.9852,  57.9627, 234.9852],
         [112.9106,  75.6363, 140.3845,  ..., 255.1328,  30.4887, 256.9644]]),
 'signer': 'Signer08',
 'text': 'mancherorts regnet es auch länger und ergiebig auch lokale überschwemmungen sind wieder möglich .'}

In [ ]:
new_train_set[7100]['sign'].shape

torch.Size([16, 432])

In [ ]:
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(new_train_set, f)

In [ ]:
!gzip -k /content/train_dataset.pkl

In [ ]:
import shutil
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-3/')
#shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-3/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/OnlyJointDataset/Attempt-3/train_dataset.pkl.gz'

# Normalization of HRNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os
import numpy as np
import torch
from pathlib import Path

In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/train.zip','/content/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/valid.zip','/content/')

In 27 node skeleton: (0-th index)

nose - 0

**shoulders - 1(l) 2(r)**

elbows - 3(l) 4(r)

**wrist - 5(l) 6(r)**

***when we normalize based on arm, center point is wrist and reference point is shoulder.***

***we normalize all hand keypoints and elbow based on arm.***

$(x',y') = (\frac{x-c_x}{d},\frac{y-c_y}{d})$

$d = \sqrt{(c_x-r_x)^2 + (c_y-r_y)^2}$ 

$(x, y)$ is the position of the point before normalization,
$(c_x, c_y)$ is the center point for calculating the relative position,
and the reference distance $d$ is the euclidean distance between
$(c_x, c_y)$ and reference point $(r_x, r_y).$


In [ ]:
import math
import os

source = '/content/train/'
dest = '/content/norm_train/'
print("Number of Input Samples: {}".format(len(os.listdir(source))))

# check if destination exists, create otherwise
path = Path(dest)
if not path.exists():
  os.makedirs(path)

count = 0
for sample in sorted(os.listdir(source)):
  whole = np.load(source + sample)
  selected = np.concatenate(([0,5,6,7,8,9,10], 
                    [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27

  #print('sample:{}'.format(sample))
  #print('whole shape:{}'.format(whole.shape))
  select = whole[:,selected,:]
  skel = np.copy(select)
  #print('skel shape:{}'.format(skel.shape))

  delta = 1e-9
  # iterate through each frame
  for idx in range(skel.shape[0]):
    # iterate through each keypoint
    for p in range(skel[idx].shape[0]):
      x, y = select[idx][p][0].item(), select[idx][p][1].item()

      if (p == 0 or p == 2 or p == 4 or p == 6) or (p >= 17 and p <= 26):
        # right side (0,2,4,6) and (17-26)
        cx, cy = select[idx][6][0].item(), select[idx][6][1].item()
        rx, ry = select[idx][2][0].item(), select[idx][2][1].item()
        d = math.sqrt((cx-rx)**2 + (cy-ry)**2)
      else:
        # left side (1,3,5) and (7-16)
        cx, cy = select[idx][6][0].item(), select[idx][6][1].item()
        rx, ry = select[idx][2][0].item(), select[idx][2][1].item()
        d = math.sqrt((cx-rx)**2 + (cy-ry)**2)

      nx, ny = (x-cx)/(d+delta) , (y-cy)/(d+delta) 
      skel[idx][p][0], skel[idx][p][1] = nx, ny 


  tensor = torch.from_numpy(skel)
  #tensor = tensor[:,:,:2]
  #tensor = torch.flatten(tensor, start_dim = 1)
  torch.save(tensor, dest + sample[:-4] + '.pt', _use_new_zipfile_serialization = False)
  count += 1
  if count % 1000 == 0:
    print('{} done!'.format(count))
print("Number of Output Samples: {}".format(len(os.listdir(dest))))

Number of Input Samples: 7096
1000 done!
2000 done!
3000 done!
4000 done!
5000 done!
6000 done!
7000 done!
Number of Output Samples: 7096


In [ ]:
import math
import os

source = '/content/dev/'
dest = '/content/norm_valid/'
print("Number of Input Samples: {}".format(len(os.listdir(source))))

# check if destination exists, create otherwise
path = Path(dest)
if not path.exists():
  os.makedirs(path)

count = 0
for sample in sorted(os.listdir(source)):
  whole = np.load(source + sample)
  selected = np.concatenate(([0,5,6,7,8,9,10], 
                    [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27

  #print('sample:{}'.format(sample))
  #print('whole shape:{}'.format(whole.shape))
  select = whole[:,selected,:]
  skel = np.copy(select)
  #print('skel shape:{}'.format(skel.shape))
  
  delta = 1e-9
  # iterate through each frame
  for idx in range(skel.shape[0]):
    # iterate through each keypoint
    for p in range(skel[idx].shape[0]):
      x, y = select[idx][p][0].item(), select[idx][p][1].item()

      if (p == 0 or p == 2 or p == 4 or p == 6) or (p >= 17 and p <= 26):
        # right side (0,2,4,6) and (17-26)
        cx, cy = select[idx][6][0].item(), select[idx][6][1].item()
        rx, ry = select[idx][2][0].item(), select[idx][2][1].item()
        d = math.sqrt((cx-rx)**2 + (cy-ry)**2)
      else:
        # left side (1,3,5) and (7-16)
        cx, cy = select[idx][6][0].item(), select[idx][6][1].item()
        rx, ry = select[idx][2][0].item(), select[idx][2][1].item()
        d = math.sqrt((cx-rx)**2 + (cy-ry)**2)

      nx, ny = (x-cx)/(d+delta) , (y-cy)/(d+delta) 
      skel[idx][p][0], skel[idx][p][1] = nx, ny 
  

  tensor = torch.from_numpy(skel)
  #tensor = tensor[:,:,:2]
  #tensor = torch.flatten(tensor, start_dim = 1)
  torch.save(tensor, dest + sample[:-4] + '.pt', _use_new_zipfile_serialization = False)
  count += 1
  if count % 100 == 0:
    print('{} done!'.format(count))
print("Number of Output Samples: {}".format(len(os.listdir(dest))))

Number of Input Samples: 519
100 done!
200 done!
300 done!
400 done!
500 done!
Number of Output Samples: 519


In [ ]:
%cd /content/norm_train/
!zip -r -q norm_train.zip ./*

/content/norm_train


In [ ]:
%cd /content/norm_valid/
!zip -r -q norm_valid.zip ./*

/content/norm_valid


In [ ]:
shutil.copy('/content/norm_train/norm_train.zip','/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/')
shutil.copy('/content/norm_valid/norm_valid.zip','/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/norm_valid.zip'

In [ ]:
source = f'/content/norm_train/'

train_set = []
count = 0

# train_data -> authors'

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 500 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([198, 54])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([92, 54])
count: 1500, sample: 15February_2010_Monday_tagesschau-1850, shape: torch.Size([93, 54])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([150, 54])
count: 2500, sample: 21April_2010_Wednesday_heute-3116, shape: torch.Size([85, 54])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([31, 54])
count: 3500, sample: 23May_2010_Sunday_tagesschau-4342, shape: torch.Size([130, 54])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([131, 54])
count: 4500, sample: 18November_2009_Wednesday_tagesschau-5589, shape: torch.Size([91, 54])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([183, 54])
count: 5500, sample: 10May_2010_Monday_tagesschau-6860, shape: torch.Size([208, 54])
count: 6000, sample: 10December_2009_Thursday_tages

In [ ]:
source = f'/content/norm_valid/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt', map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([162, 54])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([45, 54])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([65, 54])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([156, 54])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([166, 54])


In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/Normalized/temp/valid_dataset.pkl.gz'

In [ ]:
t = torch.randn((1,27,2))
t.shape

torch.Size([1, 27, 2])

In [ ]:
q = torch.flatten(t, start_dim = 1)
q.shape

torch.Size([1, 54])

In [ ]:
t

tensor([[[-0.3479,  0.0430],
         [ 1.9476,  1.3583],
         [-0.5955,  0.4441],
         [ 0.0608,  1.0643],
         [-1.3812, -0.0410],
         [-0.9868,  1.2980],
         [ 1.3730,  0.5569],
         [-0.4484,  1.4109],
         [-0.7456,  1.3498],
         [-1.9918,  0.9663],
         [-0.4871,  0.4250],
         [ 0.3840,  1.1753],
         [-0.2636, -0.5317],
         [-0.8719,  0.7928],
         [ 2.0974, -0.5921],
         [-1.4736, -1.6210],
         [ 0.4125,  0.0333],
         [ 0.3838,  1.5392],
         [-0.6992,  0.4725],
         [ 1.1557,  0.0217],
         [ 1.0335, -0.0343],
         [ 3.2318, -0.1929],
         [ 0.0388, -0.2265],
         [ 0.1840, -2.2653],
         [-0.5987, -0.9994],
         [ 0.5525,  1.2731],
         [-1.2521, -0.2543]]])

In [ ]:
q

tensor([[-0.3479,  0.0430,  1.9476,  1.3583, -0.5955,  0.4441,  0.0608,  1.0643,
         -1.3812, -0.0410, -0.9868,  1.2980,  1.3730,  0.5569, -0.4484,  1.4109,
         -0.7456,  1.3498, -1.9918,  0.9663, -0.4871,  0.4250,  0.3840,  1.1753,
         -0.2636, -0.5317, -0.8719,  0.7928,  2.0974, -0.5921, -1.4736, -1.6210,
          0.4125,  0.0333,  0.3838,  1.5392, -0.6992,  0.4725,  1.1557,  0.0217,
          1.0335, -0.0343,  3.2318, -0.1929,  0.0388, -0.2265,  0.1840, -2.2653,
         -0.5987, -0.9994,  0.5525,  1.2731, -1.2521, -0.2543]])

# wlasl

In [ ]:
!git clone https://github.com/dxli94/WLASL.git

Cloning into 'WLASL'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 216 (delta 17), reused 15 (delta 15), pack-reused 177
Receiving objects: 100% (216/216), 3.73 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
import json
with open('/content/WLASL/start_kit/WLASL_v0.3.json', 'r') as f:
  data = json.load(f)

In [ ]:
len(data)

2000

In [ ]:
type(data[0]['instances'])

list

In [ ]:
data[1113]

In [ ]:
train_ids = []
valid_ids = []
test_ids = []

id_to_label = {}

for id,entity in enumerate(data):
  for sample in entity['instances']: 
    split = sample['split']
    id_to_label[sample['video_id']] = id
    if split == 'train':
      train_ids.append(sample['video_id'])
    elif split == 'val':
      valid_ids.append(sample['video_id'])
    elif split == 'test':
      test_ids.append(sample['video_id'])

print(len(train_ids))
print(len(valid_ids))
print(len(test_ids))
print(len(id_to_label))

14289
3916
2878
21083


In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/videos.zip','/content/')

In [ ]:
!mkdir train
!mkdir valid
!mkdir test

In [ ]:
import os
from pathlib import Path

source = '/content/videos/'
print('source videos: {}'.format(len(os.listdir(source))))

for sample in os.listdir(source):
  id = sample[:-4]
  vid = source + sample

  if id in train_ids:
    shutil.copy(vid,'/content/train/')
  elif id in valid_ids:
    shutil.copy(vid, '/content/valid/')
  elif id in test_ids:
    shutil.copy(vid, '/content/test/')
    
print(len(os.listdir('/content/train/')))
print(len(os.listdir('/content/valid/')))
print(len(os.listdir('/content/test/')))    

source videos: 11980
8313
2253
1414


In [ ]:
%cd /content/train/
!zip -r -q train.zip ./*

/content/train


In [ ]:
%cd /content/valid/
!zip -r -q valid.zip ./*

/content/valid


In [ ]:
%cd /content/test/
!zip -r -q test.zip ./*

/content/test


In [ ]:
import shutil
shutil.copy('/content/train/train.zip','/content/drive/MyDrive/pose-c3d/WLASL/')
shutil.copy('/content/valid/valid.zip','/content/drive/MyDrive/pose-c3d/WLASL/')
shutil.copy('/content/test/test.zip','/content/drive/MyDrive/pose-c3d/WLASL/')

'/content/drive/MyDrive/pose-c3d/WLASL/test.zip'

In [ ]:
train_ids = os.listdir('/content/train')
valid_ids = os.listdir('/content/valid')
test_ids = os.listdir('/content/test')

print(len(train_ids)-1)
print(len(valid_ids)-1)
print(len(test_ids)-1)

8313
2253
1414


In [ ]:
import pickle

with open(r"/content/train_ids.pkl", "wb") as output_file:
  pickle.dump(train_ids, output_file)

with open(r"/content/valid_ids.pkl", "wb") as output_file:
  pickle.dump(valid_ids, output_file)

with open(r"/content/test_ids.pkl", "wb") as output_file:
  pickle.dump(test_ids, output_file)

with open(r"/content/id_to_label.pkl", "wb") as output_file:
  pickle.dump(id_to_label, output_file)

In [ ]:
import shutil
shutil.copy('/content/train_ids.pkl','/content/drive/MyDrive/pose-c3d/WLASL/')
shutil.copy('/content/valid_ids.pkl','/content/drive/MyDrive/pose-c3d/WLASL/')
shutil.copy('/content/test_ids.pkl','/content/drive/MyDrive/pose-c3d/WLASL/')
shutil.copy('/content/id_to_label.pkl','/content/drive/MyDrive/pose-c3d/WLASL/')

'/content/drive/MyDrive/pose-c3d/WLASL/id_to_label.pkl'

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/valid_pkl.zip','/content/validpkl/')

In [ ]:
import os 
pkl_list = os.listdir('/content/validpkl/valid_pkl')
print(len(pkl_list))

2184


In [ ]:
import random
valid = random.sample(pkl_list, 700)
print(len(valid))
valid

In [ ]:
src = '/content/validpkl/valid_pkl/'
dest = '/content/valid/'

for sample in valid:
  shutil.move(src + sample, dest)

train: 28
valid: 15.


In [ ]:
print('train: {}'.format(len(os.listdir(src))))
print('valid: {}.'.format(len(os.listdir(dest))))

train: 1484
valid: 700.


In [ ]:
%cd '/content/validpkl/valid_pkl'
!zip -r -q train.zip ./*

/content/validpkl/valid_pkl


In [ ]:
%cd '/content/valid/'
!zip -r -q valid.zip ./*

/content/valid


In [ ]:
shutil.copy('/content/validpkl/valid_pkl/train.zip','/content/')

'/content/train.zip'

# wlasl for posec3d

In [ ]:
import json
with open('/content/nslt_100.json', 'r') as f:
  data = json.load(f)
print(len(data))
print(type(data))

2038
<class 'dict'>


In [ ]:
data.keys()

dict_keys(['05237', '69422', '10899', '10898', '10893', '10892', '10896', '10895', '10894', '51069', '51068', '51064', '51067', '51066', '51061', '51060', '66779', '66778', '65278', '43180', '06365', '69395', '69396', '49185', '57290', '57291', '69370', '56839', '56838', '56837', '56835', '06335', '06334', '06337', '06336', '06331', '06330', '06333', '06332', '06339', '06338', '11768', '11769', '11767', '30849', '64298', '64299', '64296', '64297', '64294', '64295', '64292', '64293', '64290', '64291', '65504', '65507', '65506', '65503', '45442', '45443', '45440', '45441', '05734', '05735', '05736', '05737', '05730', '05731', '05732', '05733', '70244', '70245', '70246', '70247', '05739', '70242', '42838', '05229', '17728', '17729', '17724', '17725', '17726', '17727', '17720', '17721', '17722', '17723', '68178', '68171', '68177', '14624', '14625', '26973', '26972', '26975', '14621', '26977', '26976', '26978', '14628', '31766', '53277', '14685', '68288', '68770', '65029', '63208', '14680',

In [ ]:
train_ids = []
valid_ids = []
test_ids = []
id_to_label = {}

for key in data:
  if data[key]['subset'] == 'train':
    train_ids.append(key)
  elif data[key]['subset'] == 'val':
    valid_ids.append(key)
  elif data[key]['subset'] == 'test':
    test_ids.append(key)

  id_to_label[key] = data[key]['action'][0]

print('train:{}'.format(len(train_ids)))
print('valid:{}'.format(len(valid_ids)))
print('test:{}'.format(len(test_ids)))

train:1442
valid:338
test:258


In [ ]:
import pickle

with open(r"/content/train_ids.pkl", "wb") as output_file:
  pickle.dump(train_ids, output_file)

with open(r"/content/valid_ids.pkl", "wb") as output_file:
  pickle.dump(valid_ids, output_file)

with open(r"/content/test_ids.pkl", "wb") as output_file:
  pickle.dump(test_ids, output_file)

with open(r"/content/id_to_label.pkl", "wb") as output_file:
  pickle.dump(id_to_label, output_file)

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/valid_ids.pkl','/content/check/')
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/train_ids.pkl','/content/check/')
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/test_ids.pkl','/content/check/')

'/content/check/test_ids.pkl'

In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/train_pkl.zip','/content/check/train/')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/valid_pkl.zip','/content/check/valid/')

In [ ]:
import os

train_processed = os.listdir('/content/check/train/train_pkl')
valid_processed = os.listdir('/content/check/valid/valid_pkl')

print('train pkls processed: {}'.format(len(train_processed)))
print('valid pkls processed: {}'.format(len(valid_processed)))

train pkls processed: 2847
valid pkls processed: 700


In [ ]:
# format of name in processed files: id.pkl
# format of name in train_ids: id

# check how many train samples already processed
train_pkls_already_processed = []
valid_pkls_already_processed = []

for id in train_ids:
  name = id + '.pkl'
  if name in train_processed:
    train_pkls_already_processed.append(name)
  elif name in valid_processed:
    train_pkls_already_processed.append(name)
print('train already processed: {}'.format(len(train_pkls_already_processed)))

# check how many valid samples already processed
c = 0
for id in valid_ids:
  name = id + '.pkl'
  if name in train_processed:
    valid_pkls_already_processed.append(name)
  elif name in valid_processed:
    valid_pkls_already_processed.append(name)
print('valid already processed: {}'.format(len(valid_pkls_already_processed)))

train already processed: 131
valid already processed: 162


In [ ]:
print(len(train_pkls_already_processed))
print(len(valid_pkls_already_processed))

131
162


In [ ]:
with open(r"/content/train_processed.pkl", "wb") as output_file:
  pickle.dump(train_pkls_already_processed, output_file)

with open(r"/content/valid_processed.pkl", "wb") as output_file:
  pickle.dump(valid_pkls_already_processed, output_file)

In [ ]:
shutil.copy('/content/train_processed.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')
shutil.copy('/content/valid_processed.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')

'/content/drive/MyDrive/pose-c3d/WLASL/100/valid_processed.pkl'

In [ ]:
from os import path

for sample in train_pkls_already_processed:
  src = '/content/check/train/train_pkl/' + sample
  if path.exists(src):
    shutil.move(src ,'/content/train-wlasl-100/') 

for sample in valid_pkls_already_processed:
  src = '/content/check/valid/valid_pkl/' + sample
  if path.exists(src):
    shutil.move(src ,'/content/valid-wlasl-100/')

print('train: {}'.format(len(os.listdir('/content/train-wlasl-100'))))
print('valid: {}'.format(len(os.listdir('/content/valid-wlasl-100')))) 

train: 131
valid: 52


In [ ]:
for sample in valid_pkls_already_processed:
  src = '/content/check/train/train_pkl/' + sample
  if path.exists(src):
    shutil.move(src ,'/content/valid-wlasl-100/')

print('train: {}'.format(len(os.listdir('/content/train-wlasl-100'))))
print('valid: {}'.format(len(os.listdir('/content/valid-wlasl-100')))) 

train: 131
valid: 162


In [ ]:
%cd /content/vids/videos
!zip -r -q vids.zip ./*
shutil.copy('/content/vids/videos/vids.zip','/content/drive/MyDrive/pose-c3d/WLASL/')

/content/vids/videos


'/content/drive/MyDrive/pose-c3d/WLASL/vids.zip'

In [ ]:
%cd /content/train-wlasl-100
!zip -r -q train_pkls.zip ./*
shutil.copy('/content/train-wlasl-100/train_pkls.zip','/content/drive/MyDrive/pose-c3d/WLASL/100/')

/content/train-wlasl-100


'/content/drive/MyDrive/pose-c3d/WLASL/100/train_pkls.zip'

In [ ]:
%cd /content/valid-wlasl-100
!zip -r -q valid_pkls.zip ./*
shutil.copy('/content/valid-wlasl-100/valid_pkls.zip','/content/drive/MyDrive/pose-c3d/WLASL/100/')

/content/valid-wlasl-100


'/content/drive/MyDrive/pose-c3d/WLASL/100/valid_pkls.zip'

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/videos.zip','/content/vids/')

In [ ]:
import os
vid_list = os.listdir('/content/vids/videos')
print(len(vid_list))

11980


In [ ]:
t = 0
v = 0
tt = 0

for sample in vid_list:
  name = sample[:-4]
  if name in train_ids:
    t += 1
  elif name in valid_ids:
    v += 1
  elif name in test_ids:
    tt += 1

print('train: {}'.format(t))
print('valid: {}'.format(v))
print('test: {}'.format(tt))  

train: 748
valid: 165
test: 100


In [ ]:
for key in data:
  #print(key)
  #print(type(key))
  #print(data[key])
  print(data[key]['subset'])
  #print(data[key]['action'])
  #print(type(data[key]['action']))
  #print(data[key]['action'][0])
  #break

In [ ]:
data['10894']

{'action': [82, 1, 39], 'subset': 'train'}

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/raw_videos.zip','/content/vids/')

In [ ]:
import os

total_data = os.listdir('/content/vids/')

mp3 = []
swf = []
for sample in total_data:
  name = sample[-3:]
  if name == 'mp4':
    mp3.append(sample)
  else:
    swf.append(sample)

print('total: {}'.format(len(total_data)))
print('mp3: {}'.format(len(mp3)))
print('swf: {}'.format(len(swf)))

total: 13716
mp3: 11980
swf: 1736


In [ ]:
for sample in total_data:
  name = sample[-3:]
  if name == 'swf':
    shutil.move('/content/vids/' + sample, '/content/swf/')
print(len(os.listdir('/content/swf/')))

1736


In [ ]:
!bash swf.sh

In [ ]:
print(len(os.listdir('/content/vids')))
print(len(os.listdir('/content/swf')))
print(len(os.listdir('/content/converted')))

11980
1736
0


In [ ]:
!ffmpeg

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
src_file = '/content/swf/00.swf'
dst_file = '00340.mp4'
!ffmpeg -i {src_file} -vf pad="width=ceil(iw/2)*2:height=ceil(ih/2)*2" {dst_file}

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
for sample in os.listdir('/content/converted/'):
  shutil.move('/content/converted/' + sample,'/content/vids/')
print(len(os.listdir('/content/vids')))

In [ ]:
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/100/train_ids.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/100/train_processed.pkl','/content/')

'/content/train_processed.pkl'

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/vids.zip','/content/old_vids/')

In [ ]:
import pickle
with open('/content/train_ids.pkl', 'rb') as f:
  train_data = pickle.load(f)
with open('/content/train_processed.pkl', 'rb') as f:
  processed = pickle.load(f)

In [ ]:
old = os.listdir('/content/old_vids')

train_pending = []

for sample in old:
  id = sample[:-4]
  id2 = sample[:-3] + 'pkl'
  if id in train_data and id2 not in processed:
    train_pending.append(sample)

print('pending:{}'.format(len(train_pending)))  

pending:617


In [ ]:
with open('/content/train_pending.pkl', 'wb') as f:
  pickle.dump(train_pending, f)

In [ ]:
shutil.copy('/content/train_pending.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')

'/content/drive/MyDrive/pose-c3d/WLASL/100/train_pending.pkl'

In [ ]:
import json
with open('/content/nslt_300.json', 'r') as f:
  data2 = json.load(f)

In [ ]:
len(data2)

5118

In [ ]:
!sudo apt-get install ffmpeg swftools

In [ ]:
!swfcombine -d /content/57290.swf -o /content/57290-new.swf

FATAL   Failed to read from /content/57290.swf


In [ ]:
import json

with open('/content/asl100.json', 'rb') as f:
    data = json.load(f)

In [ ]:
data[0]

In [ ]:
import json

with open('/content/WLASL_v0.3.json', 'rb') as f:
    data2 = json.load(f)
data2[0]

In [ ]:
import json

with open('/content/image_00001_keypoints.json', 'rb') as f:
    data = json.load(f)
print(type(data))

<class 'dict'>


In [ ]:
len(data)

2

In [ ]:
data.keys()

dict_keys(['version', 'people'])

In [ ]:
data['version']

1.3

In [ ]:
data['people'][0]['hand_left_keypoints_2d']

In [ ]:
len(data['people'][0]['hand_left_keypoints_2d'])

63

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/100/train_ids.pkl','/content/')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/100/train_pkls.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/WLASL2000.zip','/content/vids/')

In [ ]:
import pickle

with open('/content/train_ids.pkl', 'rb') as f:
    train_ids = pickle.load(f)
print(len(train_ids))

1442


In [ ]:
import os
train_proc = os.listdir('/content/train/train_pkls/')
len(train_proc)

734

In [ ]:
src = os.listdir('/content/vids/')
print('total:{}'.format(len(src)))

train_pending = []

for sample in src:
  id = sample[:-4]
  name = sample[:-4] + '.pkl' 
  if id in train_ids and name not in train_proc:
    train_pending.append(id)
print(len(train_pending))

total:21095
708


In [ ]:
with open('/content/train_pending.pkl', 'wb') as f:
    pickle.dump(train_pending, f)

In [ ]:
shutil.copy('/content/train_pending.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')

'/content/drive/MyDrive/pose-c3d/WLASL/100/train_pending.pkl'

In [ ]:
shutil.copy('/content/drive/MyDrive/pose-c3d/WLASL/100/valid_ids.pkl','/content/')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/100/valid_pkls.zip','/content/valid/')

In [ ]:
import pickle

with open('/content/valid_ids.pkl', 'rb') as f:
    valid_ids = pickle.load(f)
print(len(valid_ids))

338


In [ ]:
import os
valid_proc = os.listdir('/content/valid/')
len(valid_proc)

162

In [ ]:
src = os.listdir('/content/vids/')
print('total:{}'.format(len(src)))

valid_pending = []

for sample in src:
  id = sample[:-4]
  name = sample[:-4] + '.pkl' 
  if id in valid_ids and name not in valid_proc:
    valid_pending.append(id)
print(len(valid_pending))

total:21095
176


In [ ]:
with open('/content/valid_pending.pkl', 'wb') as f:
    pickle.dump(valid_pending, f)

In [ ]:
shutil.copy('/content/valid_pending.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')

'/content/drive/MyDrive/pose-c3d/WLASL/100/valid_pending.pkl'

# wlasl - combine pkls

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/100/train_pkls.zip','/content/train-pkls')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/WLASL/100/valid_pkls.zip','/content/valid-pkls')

In [ ]:
import os
import pickle

wlasl100_train = []
wlasl100_valid = []

s = '/content/train-pkls/train_pkls/'
v = '/content/valid-pkls/valid_pkls/'

tsrc = os.listdir(s)
vsrc = os.listdir(v)

for sample in tsrc:
  loc = s + sample
  
  with open(loc, 'rb') as f:
    pkl = pickle.load(f)

  wlasl100_train.append(pkl)

for sample in vsrc:
  loc = v + sample

  with open(loc, 'rb') as f:
    pkl = pickle.load(f)

  wlasl100_valid.append(pkl)

print(len(wlasl100_train))
print(len(wlasl100_valid))

1402
327


In [ ]:
with open('/content/train-wlasl100.pkl', 'wb') as f:
    pickle.dump(wlasl100_train, f)
with open('/content/valid-wlasl100.pkl', 'wb') as f:
    pickle.dump(wlasl100_valid, f)

In [ ]:
shutil.copy('/content/train-wlasl100.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')
shutil.copy('/content/valid-wlasl100.pkl','/content/drive/MyDrive/pose-c3d/WLASL/100/')

'/content/drive/MyDrive/pose-c3d/WLASL/100/valid-wlasl100.pkl'

# AUTSL dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**generate id to label dictionary for train set**

In [ ]:
import csv

id_to_label = {}

with open('/content/train_labels.csv', mode='r') as infile:
    reader = csv.reader(infile)
    line_count = 0
    for rows in reader:
      line_count += 1
      id_to_label[rows[0]] = rows[1]

print(len(id_to_label))
print(line_count)

28142
28142


In [ ]:
import csv

id_to_label_val = {}

with open('/content/ground_truth.csv', mode='r') as infile:
    reader = csv.reader(infile)
    line_count = 0
    for rows in reader:
      line_count += 1
      id_to_label_val[rows[0]] = rows[1]

print(len(id_to_label_val))
print(line_count)

4418
4418


In [ ]:
train_id = []
for id in id_to_label.keys():
  train_id.append(id)

len(train_id) 

28142

In [ ]:
train_id = []
for id in id_to_label.keys():
  train_id.append(id)

len(train_id) 

In [ ]:
!mkdir train

In [ ]:
subset = train_id[25000:28412]

count = 0
for id in subset:
  loc = '/content/drive/.shortcut-targets-by-id/1tFu1ysY_OHFIQTnmGzCRjRwOxWWUQQbR/Dataset/train_npy_files_Darkpose/' + id + '_color.npy'
  shutil.copy(loc, '/content/train/')
  count += 1
  if count % 100 == 0:
    print('progress:{}'.format(count))

print(len(os.listdir('/content/train/')))

In [ ]:
%cd /content/valid
!zip -r -y valid-133-darkpose.zip ./*

In [ ]:
%cd /content/train
!zip -r -q -y train-133-darkpose.zip ./*

/content/train


In [ ]:
len(os.listdir('/content/valid'))

4419

In [ ]:
import shutil
shutil.copy('/content/train/train-133-darkpose.zip','/content/drive/MyDrive/AUTSL/')
shutil.copy('/content/valid/valid-133-darkpose.zip','/content/drive/MyDrive/AUTSL/')

'/content/drive/MyDrive/AUTSL/valid-133-darkpose.zip'

In [ ]:
import os
os.remove('/content/train/train-133-darkpose.zip')
os.remove('/content/valid/valid-133-darkpose.zip')

In [ ]:
%cd /content/

/content


In [ ]:
len(id_to_label)

28142

**prepare data for posec3d**

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/AUTSL/train-133-darkpose.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/AUTSL/valid-133-darkpose.zip','/content/valid/')

In [ ]:
import os
print(len(os.listdir('/content/train')))
print(len(os.listdir('/content/valid')))

28142
4418


In [ ]:
import numpy as np 
import os

src = '/content/valid/'
src_list = os.listdir(src)
valid_list = []
count = 0

for sample in src_list:

  name = src + sample
  whole = np.load(name)

  #joints = np.array([0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132])
  joints = np.concatenate(([0,5,6,7,8,9,10], 
                    [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27
  
  select = whole[:,joints,:]
  skeleton = np.copy(select)

  #print(whole.shape)
  
  kpts = np.copy(skeleton)
  kpts = kpts[:,:,:2]
  kpts = np.expand_dims(kpts,0)

  #print(kpts.shape)

  conf = np.copy(skeleton)
  conf = conf[:,:,2:]
  conf = np.expand_dims(conf,0)

  #print(conf.shape)

  anno = dict()
  anno['keypoint'] = kpts
  anno['keypoint_score'] = conf
  anno['frame_dir'] = sample[:-10]
  anno['img_shape'] = (512, 512)
  anno['original_shape'] = (512, 512)
  anno['total_frames'] = whole.shape[0]
  anno['label'] = int(id_to_label_val[sample[:-10]])

  #print(anno)
  #break

  valid_list.append(anno)
  count += 1

  if count % 1000 == 0:
    print(count)

print('total:{}'.format(len(valid_list)))

1000
2000
3000
4000
total:4418


In [ ]:
import numpy as np 
import os

src = '/content/train/'
src_list = os.listdir(src)
train_list = []
count = 0

for sample in src_list:

  name = src + sample
  whole = np.load(name)

  #joints = np.array([0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132])
  joints = np.concatenate(([0,5,6,7,8,9,10], 
                    [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27
  
  select = whole[:,joints,:]
  skeleton = np.copy(select)

  #print(whole.shape)
  
  kpts = np.copy(skeleton)
  kpts = kpts[:,:,:2]
  kpts = np.expand_dims(kpts,0)

  #print(kpts.shape)

  conf = np.copy(skeleton)
  conf = conf[:,:,2:]
  conf = np.expand_dims(conf,0)

  #print(conf.shape)

  anno = dict()
  anno['keypoint'] = kpts
  anno['keypoint_score'] = conf
  anno['frame_dir'] = sample[:-10]
  anno['img_shape'] = (512, 512)
  anno['original_shape'] = (512, 512)
  anno['total_frames'] = whole.shape[0]
  anno['label'] = int(id_to_label[sample[:-10]])

  #print(anno)
  #break

  train_list.append(anno)
  count += 1

  if count % 1000 == 0:
    print(count)

print('total:{}'.format(len(train_list)))

In [ ]:
import pickle
with open('/content/train-autsl.pkl', 'wb') as f:
    pickle.dump(train_list, f)
with open('/content/valid-autsl.pkl', 'wb') as f:
    pickle.dump(valid_list, f)

In [ ]:
shutil.copy('/content/train-autsl.pkl','/content/drive/MyDrive/pose-c3d/AUTSL/27kpts/')
shutil.copy('/content/valid-autsl.pkl','/content/drive/MyDrive/pose-c3d/AUTSL/27kpts/')

'/content/drive/MyDrive/pose-c3d/AUTSL/27kpts/valid-autsl.pkl'

left - **0**, 5, 7, 9, 91, 95, 96, 99, 100, 103, 104, 107, 108, 111.

right - 6, 8, 10, 112, 116, 117, 120, 121, 124, 125, 128, 129, 132.

total - 0,5,6,7,8,9,10,91,95,96,99,100,103,104,107,108,111,112,116,117,120,121,124,125,128,129,132.

left - 0, 5, 7, 91, 94, 99, 103, 107, 111

right - 6, 8, 112, 116, 120, 124, 128, 132

total - 0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132

In [ ]:
'''
whole = np.load(source + sample)
selected = np.concatenate(([0,5,6,7,8,9,10], 
                  [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27

#print('sample:{}'.format(sample))
#print('whole shape:{}'.format(whole.shape))
select = whole[:,selected,:]
skel = np.copy(select)
#print('skel shape:{}'.format(skel.shape))
'''

# Phoenix-2014T dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/train.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/HRNet-133/valid.zip','/content/valid/')

In [ ]:
import os

val = os.listdir('/content/train/train/')
train = os.listdir('/content/valid/dev/')
print(len(val))
print(len(train))

7096
519


**prepare data for posec3d**

In [ ]:
import numpy as np 
import os

src = '/content/valid/dev/'
src_list = os.listdir(src)
raw_valid_list = []
valid_endpoints = []
count = 0


for sample in src_list:

  name = src + sample
  whole = np.load(name)

  #print(whole.shape)

  joints = np.array([0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132])
  select = whole[:,joints,:]
  skeleton = np.copy(select)

  for frame in range(0, skeleton.shape[0], 8):
    if frame + 8 <= skeleton.shape[0]:
      subset = skeleton[frame:frame + 8, :, :]
    else:
      subset = skeleton[frame:, :, :]
      diff = 8 - subset.shape[0]
      x = subset[-1]
      arrays = [x for _ in range(diff)]
      temp = np.stack(arrays, axis=0)
      subset = np.concatenate((subset, temp), axis = 0)
    

    #print(subset.shape)

    kpts = np.copy(subset)
    kpts = kpts[:,:,:2]
    kpts = np.expand_dims(kpts,0)

    #print(kpts.shape)

    conf = np.copy(subset)
    conf = conf[:,:,2:]
    conf = np.expand_dims(conf,0)

    #print(conf.shape)
    

    anno = dict()
    anno['keypoint'] = kpts
    anno['keypoint_score'] = conf
    anno['frame_dir'] = sample[:-4]
    anno['img_shape'] = (210, 260)
    anno['original_shape'] = (210, 260)
    anno['total_frames'] = int(8)
    anno['label'] = int(-1)

  #print(anno)

    raw_valid_list.append(anno)
    
  count += 1
  valid_endpoints.append(int(len(raw_valid_list) - 1))
  
  if count % 1000 == 0:
    print(count)

print('total:{}'.format(len(valid_endpoints)))

total:519


In [ ]:
import numpy as np 
import os

src = '/content/train/train/'
src_list = os.listdir(src)
raw_train_list = []
train_endpoints = []
count = 0


for sample in src_list:

  name = src + sample
  whole = np.load(name)

  #print(whole.shape)

  joints = np.array([0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132])
  select = whole[:,joints,:]
  skeleton = np.copy(select)

  for frame in range(0, skeleton.shape[0], 8):
    if frame + 8 <= skeleton.shape[0]:
      subset = skeleton[frame:frame + 8, :, :]
    else:
      subset = skeleton[frame:, :, :]
      diff = 8 - subset.shape[0]
      x = subset[-1]
      arrays = [x for _ in range(diff)]
      temp = np.stack(arrays, axis=0)
      subset = np.concatenate((subset, temp), axis = 0)
    

    #print(subset.shape)

    kpts = np.copy(subset)
    kpts = kpts[:,:,:2]
    kpts = np.expand_dims(kpts,0)

    #print(kpts.shape)

    conf = np.copy(subset)
    conf = conf[:,:,2:]
    conf = np.expand_dims(conf,0)

    #print(conf.shape)
    

    anno = dict()
    anno['keypoint'] = kpts
    anno['keypoint_score'] = conf
    anno['frame_dir'] = sample[:-4]
    anno['img_shape'] = (210, 260)
    anno['original_shape'] = (210, 260)
    anno['total_frames'] = int(8)
    anno['label'] = int(-1)

  #print(anno)

    raw_train_list.append(anno)
    
  count += 1
  train_endpoints.append(int(len(raw_train_list) - 1))
  
  if count % 1000 == 0:
    print(count)

print('total:{}'.format(len(train_endpoints)))

1000
2000
3000
4000
5000
6000
7000
total:7096


In [ ]:
import pickle
with open('/content/train-endpoints.pkl', 'wb') as f:
    pickle.dump(train_endpoints, f)
with open('/content/valid-endpoints.pkl', 'wb') as f:
    pickle.dump(valid_endpoints, f)

with open('/content/train-phoenix2014t.pkl', 'wb') as f:
    pickle.dump(raw_train_list, f)
with open('/content/valid-phoenix2014t.pkl', 'wb') as f:
    pickle.dump(raw_valid_list, f)

In [ ]:
import shutil
shutil.copy('/content/train-endpoints.pkl','/content/drive/MyDrive/pose-c3d/Phoenix2014T/')
shutil.copy('/content/valid-endpoints.pkl','/content/drive/MyDrive/pose-c3d/Phoenix2014T/')
shutil.copy('/content/train-phoenix2014t.pkl','/content/drive/MyDrive/pose-c3d/Phoenix2014T/')
shutil.copy('/content/valid-phoenix2014t.pkl','/content/drive/MyDrive/pose-c3d/Phoenix2014T/')

'/content/drive/MyDrive/pose-c3d/Phoenix2014T/valid-phoenix2014t.pkl'

In [ ]:
import shutil

In [ ]:
import pickle
with open('/content/anno.pkl', 'wb') as f:
    pickle.dump(anno, f)

In [ ]:
import pickle
with open('/content/train-autsl.pkl', 'wb') as f:
    pickle.dump(train_list, f)
with open('/content/valid-autsl.pkl', 'wb') as f:
    pickle.dump(valid_list, f)

In [ ]:
shutil.copy('/content/train-autsl.pkl','/content/drive/MyDrive/pose-c3d/AUTSL/')
shutil.copy('/content/valid-autsl.pkl','/content/drive/MyDrive/pose-c3d/AUTSL/')

'/content/drive/MyDrive/pose-c3d/AUTSL/valid-autsl.pkl'

left - 0, 5, 7, 91, 94, 99, 103, 107, 111

right - 6, 8, 112, 116, 120, 124, 128, 132

total - 0,5,6,7,8,91,94,99,103,107,111,112,116,120,124,128,132

In [ ]:
'''
whole = np.load(source + sample)
selected = np.concatenate(([0,5,6,7,8,9,10], 
                  [91,95,96,99,100,103,104,107,108,111],[112,116,117,120,121,124,125,128,129,132]), axis=0) #27

#print('sample:{}'.format(sample))
#print('whole shape:{}'.format(whole.shape))
select = whole[:,selected,:]
skel = np.copy(select)
#print('skel shape:{}'.format(skel.shape))
'''

In [ ]:
import pickle
with open('/content/result.pkl','rb') as f:
  res = pickle.load(f)
type(res)

list

In [ ]:
len(res)

327

In [ ]:
type(res[0])

numpy.ndarray

In [ ]:
res[0].shape

(512,)

In [ ]:
import shutil

shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/valid-result-raw.pkl','/content/')

shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-0k-3k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-3k-6k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-6k-8k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-8k-10k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-10k-12k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-12k-15k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-15k-16k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-16k-20k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-20k-25k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-25k-30k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-30k-35k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-35k-40k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-40k-50k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-50k-60k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-60k-70k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-70k-80k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-80k-90k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-90k-100k.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train-result-raw-100k-106k.pkl','/content/')

shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/train-endpoints.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/valid-endpoints.pkl','/content/')

shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/train-phoenix2014t.pkl','/content/')
shutil.copy('/content/drive/MyDrive/pose-c3d/Phoenix2014T/valid-phoenix2014t.pkl','/content/')

'/content/valid-phoenix2014t.pkl'

In [ ]:
import pickle

with open('/content/valid-result-raw.pkl','rb') as f:
  valid = pickle.load(f)
print(len(valid))
print(valid[0].shape)

with open('/content/valid-endpoints.pkl','rb') as f:
  valid_endpoints = pickle.load(f)
print(len(valid_endpoints))

with open('/content/valid-phoenix2014t.pkl','rb') as f:
  valid_ori = pickle.load(f)
print(len(valid_ori))

7196
(512,)
519
7196


In [ ]:
!mkdir valid

In [ ]:
import torch
a = torch.randn((512))

b = torch.randn((512))

c = torch.stack((a,b),0)
c.shape

torch.Size([2, 512])

In [ ]:
for idx, sample in enumerate(valid):
  print(sample)
  break

In [ ]:
for idx,sample in enumerate(valid_ori):
  print(idx, sample['frame_dir'])
  if idx == 50:
    break

In [ ]:
print(valid_endpoints[0])
print(valid_endpoints[-1])
print(len(valid_endpoints))

15
7195
519


In [ ]:
shutil.rmtree('/content/valid/')
!mkdir valid

In [ ]:
import torch
import os

target = '/content/valid/'
counter = 0
flag = False

tensor_list = []

for idx, sample in enumerate(valid):

  if idx in valid_endpoints:
    tensor = torch.from_numpy(sample)
    tensor_list.append(tensor)
    
    feature = torch.stack(tensor_list, axis = 0)
    torch.save(feature, target + valid_ori[idx]['frame_dir'] + '.pt', _use_new_zipfile_serialization = False)
    tensor_list.clear()
    counter += 1
    if counter % 50 == 0:
      print(counter, feature.shape)
  else:
    tensor = torch.from_numpy(sample)
    tensor_list.append(tensor)

print(len(os.listdir(target)))

50 torch.Size([14, 512])
100 torch.Size([11, 512])
150 torch.Size([19, 512])
200 torch.Size([14, 512])
250 torch.Size([9, 512])
300 torch.Size([14, 512])
350 torch.Size([12, 512])
400 torch.Size([12, 512])
450 torch.Size([15, 512])
500 torch.Size([14, 512])
519


In [ ]:
%cd /content/valid/
!zip -r -q valid.zip ./*

/content/valid


In [ ]:
shutil.copy('/content/valid/valid.zip','/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts')

'/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/valid.zip'

In [ ]:
import pickle
with open('/content/train-result-raw-0k-3k.pkl','rb') as f:
  t1 = pickle.load(f)

with open('/content/train-result-raw-3k-6k.pkl','rb') as f:
  t2 = pickle.load(f)

with open('/content/train-result-raw-6k-8k.pkl','rb') as f:
  t3 = pickle.load(f)

with open('/content/train-result-raw-8k-10k.pkl','rb') as f:
  t4 = pickle.load(f)

with open('/content/train-result-raw-10k-12k.pkl','rb') as f:
  t5 = pickle.load(f)

with open('/content/train-result-raw-12k-15k.pkl','rb') as f:
  t6 = pickle.load(f)

with open('/content/train-result-raw-15k-16k.pkl','rb') as f:
  t7 = pickle.load(f)

with open('/content/train-result-raw-16k-20k.pkl','rb') as f:
  t8 = pickle.load(f)

with open('/content/train-result-raw-20k-25k.pkl','rb') as f:
  t9 = pickle.load(f)

with open('/content/train-result-raw-25k-30k.pkl','rb') as f:
  t10 = pickle.load(f)

with open('/content/train-result-raw-30k-35k.pkl','rb') as f:
  t11 = pickle.load(f)

with open('/content/train-result-raw-35k-40k.pkl','rb') as f:
  t12 = pickle.load(f)

with open('/content/train-result-raw-40k-50k.pkl','rb') as f:
  t13 = pickle.load(f)

with open('/content/train-result-raw-50k-60k.pkl','rb') as f:
  t14 = pickle.load(f)

with open('/content/train-result-raw-60k-70k.pkl','rb') as f:
  t15 = pickle.load(f)

with open('/content/train-result-raw-70k-80k.pkl','rb') as f:
  t16 = pickle.load(f)

with open('/content/train-result-raw-80k-90k.pkl','rb') as f:
  t17 = pickle.load(f)

with open('/content/train-result-raw-90k-100k.pkl','rb') as f:
  t18 = pickle.load(f)

with open('/content/train-result-raw-100k-106k.pkl','rb') as f:
  t19 = pickle.load(f)

In [ ]:
import itertools

train = []

for i in itertools.chain(t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19):
  train.append(i) 
print(len(train))

106536


In [ ]:
print(train[0].shape)

with open('/content/train-endpoints.pkl','rb') as f:
  train_endpoints = pickle.load(f)
print(len(train_endpoints))

with open('/content/train-phoenix2014t.pkl','rb') as f:
  train_ori = pickle.load(f)
print(len(train_ori))

(512,)
7096
106536


In [ ]:
#shutil.rmtree('/content/valid/')
%cd /content/
!mkdir train

/content


In [ ]:
import torch
import os

target = '/content/train/'
counter = 0
flag = False

tensor_list = []

for idx, sample in enumerate(train):

  if idx in train_endpoints:
    tensor = torch.from_numpy(sample)
    tensor_list.append(tensor)
    
    feature = torch.stack(tensor_list, axis = 0)
    torch.save(feature, target + train_ori[idx]['frame_dir'] + '.pt', _use_new_zipfile_serialization = False)
    tensor_list.clear()
    counter += 1
    if counter % 500 == 0:
      print(counter, feature.shape)
  else:
    tensor = torch.from_numpy(sample)
    tensor_list.append(tensor)

print(len(os.listdir(target)))

500 torch.Size([7, 512])
1000 torch.Size([3, 512])
1500 torch.Size([9, 512])
2000 torch.Size([23, 512])
2500 torch.Size([30, 512])
3000 torch.Size([17, 512])
3500 torch.Size([25, 512])
4000 torch.Size([16, 512])
4500 torch.Size([18, 512])
5000 torch.Size([11, 512])
5500 torch.Size([6, 512])
6000 torch.Size([25, 512])
6500 torch.Size([23, 512])
7000 torch.Size([14, 512])
7096


In [ ]:
%cd /content/train/
!zip -r -q train.zip ./*

/content/train


In [ ]:
shutil.copy('/content/train/train.zip','/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts')

'/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train.zip'

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/train.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/valid.zip','/content/valid/')

In [ ]:
source = f'/content/train/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt') # map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  train_set.append(temp_dict)
  count += 1
  if count % 500 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))
print(len(train_set))    

count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([25, 512])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([12, 512])
count: 1500, sample: 15February_2010_Monday_tagesschau-1850, shape: torch.Size([12, 512])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([19, 512])
count: 2500, sample: 21April_2010_Wednesday_heute-3116, shape: torch.Size([11, 512])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([4, 512])
count: 3500, sample: 23May_2010_Sunday_tagesschau-4342, shape: torch.Size([17, 512])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([17, 512])
count: 4500, sample: 18November_2009_Wednesday_tagesschau-5589, shape: torch.Size([12, 512])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([23, 512])
count: 5500, sample: 10May_2010_Monday_tagesschau-6860, shape: torch.Size([26, 512])
count: 6000, sample: 10December_2009_Thursday_t

In [ ]:
source = f'/content/valid/'

valid_set = []
count = 0

# valid_data -> authors'

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  tensor = torch.load(source + name + '.pt') # map_location = torch.device('cpu'))
   
  temp_dict = sample
  temp_dict['sign'] = tensor

  valid_set.append(temp_dict)
  count += 1
  if count % 100 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))
print(len(valid_set))     

count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([21, 512])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([6, 512])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([9, 512])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([20, 512])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([21, 512])
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/')

'/content/drive/MyDrive/pose-c3d/Phoenix2014T/17kpts/valid_dataset.pkl.gz'

# Pretrained Transformers

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/train_dataset.pkl.gz','/content/')
shutil.copy('/content/drive/MyDrive/SLT-2020/Joint_Dataset/8_Finetuned/valid_dataset.pkl.gz','/content/')

'/content/valid_dataset.pkl.gz'

In [ ]:
train = load_dataset_file('/content/train_dataset.pkl.gz')
print(type(train))

<class 'list'>


In [ ]:
train[0]['sign'].shape

torch.Size([11, 1152])

# pose-c3d

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/pose-c3d/ntu60_xsub_val.pkl','/content/')

'/content/ntu60_xsub_val.pkl'

In [ ]:
import pickle

with open('/content/ntu60_xsub_val.pkl', 'rb') as f:
    data = pickle.load(f)
print(type(data))
print(len(data))
print(type(data[0]))

<class 'list'>
16487
<class 'dict'>


In [ ]:
data[0]

In [ ]:
data[0]['keypoint'].shape # -> #person x #frames x #keypoints x #x,y 

(1, 63, 17, 2)

In [ ]:
data[0]['keypoint_score'].shape # -> #person x #frames x #confidence

(1, 63, 17)

# SL-GCN Joint 6 Frames

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/6_Finetuned/train.zip','/content/train/')
shutil.unpack_archive('/content/drive/MyDrive/SLT-2020/Joint_Dataset/6_Finetuned/valid.zip','/content/valid/')

In [ ]:
train_data[0]

{'gloss': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'name': 'train/11August_2010_Wednesday_tagesschau-1',
 'sign': tensor([[0.0998, 0.0000, 0.1679,  ..., 0.0000, 0.0000, 0.0000],
         [4.6956, 0.0000, 0.2897,  ..., 0.0000, 0.0000, 0.0000],
         [6.8340, 0.0492, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [9.7269, 0.4355, 0.0000,  ..., 3.4021, 0.0000, 0.0000],
         [8.1934, 0.3163, 0.0000,  ..., 0.0378, 0.0000, 0.0000],
         [6.4664, 0.1839, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'signer': 'Signer08',
 'text': 'und nun die wettervorhersage für morgen donnerstag den zwölften august .'}

In [ ]:
source = f'/content/train/'

train_set = []
count = 0

for sample in train_data:
  name = sample['name'][6:] # train -> [6:] valid -> [4:]
  np_file = np.load(source + name + '.npy')
  tensor_file = torch.from_numpy(np_file)
  tensor_file = tensor_file.squeeze(1).reshape(tensor_file.shape[0], 1152, -1)
  #print(tensor_file.shape)
  tensor_file = tensor_file.mean(2)
  #print(tensor_file.shape)
  #break
  

  temp_dict = sample
  temp_dict['sign'] = tensor_file

  train_set.append(temp_dict)
  count += 1
  if count % 500 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 500, sample: 18October_2010_Monday_heute-626, shape: torch.Size([33, 1152])
count: 1000, sample: 02August_2010_Monday_heute-1248, shape: torch.Size([16, 1152])
count: 1500, sample: 15February_2010_Monday_tagesschau-1850, shape: torch.Size([16, 1152])
count: 2000, sample: 15September_2010_Wednesday_heute-2474, shape: torch.Size([25, 1152])
count: 2500, sample: 21April_2010_Wednesday_heute-3116, shape: torch.Size([15, 1152])
count: 3000, sample: 24January_2011_Monday_tagesschau-3717, shape: torch.Size([6, 1152])
count: 3500, sample: 23May_2010_Sunday_tagesschau-4342, shape: torch.Size([22, 1152])
count: 4000, sample: 25April_2010_Sunday_tagesschau-4947, shape: torch.Size([22, 1152])
count: 4500, sample: 18November_2009_Wednesday_tagesschau-5589, shape: torch.Size([16, 1152])
count: 5000, sample: 18July_2010_Sunday_tagesschau-6225, shape: torch.Size([31, 1152])
count: 5500, sample: 10May_2010_Monday_tagesschau-6860, shape: torch.Size([35, 1152])
count: 6000, sample: 10December_2009

In [ ]:
source = f'/content/valid/'

valid_set = []
count = 0

for sample in valid_data:
  name = sample['name'][4:] # train -> [6:] valid -> [4:]
  np_file = np.load(source + name + '.npy')
  tensor_file = torch.from_numpy(np_file)
  tensor_file = tensor_file.squeeze(1).reshape(tensor_file.shape[0], 1152, -1)
  #print(tensor_file.shape)
  tensor_file = tensor_file.mean(2)

  temp_dict = sample
  temp_dict['sign'] = tensor_file

  valid_set.append(temp_dict)
  count += 1
  if count % 50 == 0:
    print('count: {}, sample: {}, shape: {}'.format(count, name, temp_dict['sign'].shape))    

count: 50, sample: 03February_2011_Thursday_tagesschau-938, shape: torch.Size([18, 1152])
count: 100, sample: 09March_2011_Wednesday_heute-1742, shape: torch.Size([27, 1152])
count: 150, sample: 25November_2010_Thursday_tagesschau-2529, shape: torch.Size([17, 1152])
count: 200, sample: 27August_2009_Thursday_tagesschau-3284, shape: torch.Size([8, 1152])
count: 250, sample: 26August_2010_Thursday_tagesschau-4265, shape: torch.Size([19, 1152])
count: 300, sample: 23August_2010_Monday_heute-5319, shape: torch.Size([11, 1152])
count: 350, sample: 24September_2009_Thursday_heute-6077, shape: torch.Size([28, 1152])
count: 400, sample: 06July_2011_Wednesday_tagesschau-6870, shape: torch.Size([26, 1152])
count: 450, sample: 03October_2012_Wednesday_tagesschau-7651, shape: torch.Size([17, 1152])
count: 500, sample: 28July_2011_Thursday_tagesschau-8521, shape: torch.Size([28, 1152])


In [ ]:
print(len(train_set))
print(len(valid_set))

7096
519


In [ ]:
with open('/content/valid_dataset.pkl', 'wb') as f:
  pickle.dump(valid_set, f)
with open('/content/train_dataset.pkl', 'wb') as f:
  pickle.dump(train_set, f)

In [ ]:
!gzip -k /content/valid_dataset.pkl
!gzip -k /content/train_dataset.pkl

In [ ]:
shutil.copy('/content/train_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/6_Finetuned/1152/')
shutil.copy('/content/valid_dataset.pkl.gz','/content/drive/MyDrive/SLT-2020/Joint_Dataset/6_Finetuned/1152/')

'/content/drive/MyDrive/SLT-2020/Joint_Dataset/6_Finetuned/1152/valid_dataset.pkl.gz'